# *Question 1*: How well do the different reactivation initiatives work?

## (1st Approach - Using the Data as it is)

**Assumptions**

* In the **"3_emails_take_home"** file, the column *email_type* consists of 4 different values/entries namely **reactivation**, **update**,**newsletter** and **promo**. I have taken into account only the value **reactivation**. The other 3 values are not interesting enough for considering it into our analysis as it does not include any outreach measures for reactivation. 

* I have considered that the given dataset sample among the 4 csv files as being real and they contain the actual/clean data.

### Interpretation/Answer

The reactivation initiative by **Call** seems to be the more effective method in trying to win back the customers who have cancelled their recurring bookings. The success rate of the Call initiative is around **58%**, whereas that of Email stands at **39%** and control group initiative at **19%**. 

In [54]:
import pandas as pd
import sqlite3

# Load the CSV data into a DataFrame
df_cancellations = pd.read_csv(r'C:\Users\Rajatth\Downloads\4_cancellations_take_home.csv')
df_emails = pd.read_csv(r'C:\Users\Rajatth\Downloads\3_emails_take_home.csv')
df_calls = pd.read_csv(r'C:\Users\Rajatth\Downloads\2_calls_take_home.csv')
df_appointments = pd.read_csv(r'C:\Users\Rajatth\Downloads\1_cleaning_appointments_take_home.csv')

# Create a SQLite database and connect to it
conn = sqlite3.connect(':memory:')  # This creates a new database in RAM

# Convert the DataFrame into a SQL table
df_cancellations.to_sql('cancellations', conn, if_exists='replace', index=False)
df_calls.to_sql('calls', conn, if_exists='replace', index=False)
df_emails.to_sql('emails', conn, if_exists='replace', index=False)
df_appointments.to_sql('appointments', conn, if_exists='replace', index=False)

# SQL query
query = """
-- Grouping each reactivation strategy into categories. 
WITH Initiative_Method AS (
    SELECT 
        can.customer_id, 
        CASE 
            WHEN EXISTS (SELECT 1 FROM emails e WHERE e.customer_id = can.customer_id AND email_type = 'reactivation') THEN 'email'
            WHEN EXISTS (SELECT 1 FROM calls c WHERE c.customer_id = can.customer_id) THEN 'call' 
            ELSE 'control'
        END AS initiative_method 
    FROM cancellations can
),


-- Identifying the customers who were won back and who were not
reactivated_customers AS (
    SELECT 
        can.customer_id,
        CASE
            WHEN EXISTS (SELECT 1 
                         FROM appointments app 
                         WHERE app.customer_id = can.customer_id
                           AND appointment_timestamp > cancellation_timestamp) 
                THEN 'reactivated'
            ELSE 'not_reactivated'
        END AS reactivation_status
    FROM cancellations can
)


-- Counting the total number of reactivated customers and their success rate with respect to total customers
SELECT 
    initiative_method,
    COUNT(*) AS total_customers_attempted,
    SUM(CASE 
            WHEN reactivation_status = 'reactivated' THEN 1 
            ELSE 0 
        END) AS reactivated_customers,
    (SUM(CASE 
             WHEN reactivation_status = 'reactivated' THEN 1 
             ELSE 0 
        END) * 100.0 / COUNT(*)) AS success_rate
FROM Initiative_Method
LEFT JOIN reactivated_customers USING (customer_id)
GROUP BY initiative_method;
"""

# Execute the SQL query and fetch the results into a new DataFrame
result_df = pd.read_sql_query(query, conn)

# Display the result
print(result_df)

  initiative_method  total_customers_attempted  reactivated_customers  \
0              call                       1684                    974   
1           control                       1622                    303   
2             email                       1694                    663   

   success_rate  
0     57.838480  
1     18.680641  
2     39.138135  


# *Question 1*: How well do the different reactivation initiatives work?

## (2nd Approach - Removing the fake data and utilizing valid data)

**Assumptions**

* In the **"3_emails_take_home"** file, the column *email_type* consists of 4 different values/entries. I have taken into account only the value **reactivation**. The other 3 values are not interesting enough for considering it into our analysis as it does not include any outreach measures for reactivation. 

* I have optimized the query to include only the possible real data and eliminate the entries which are fake/not factual. 

* **Ex:** Once the booking has been cancelled by the customer on a particular date, new appointment date falls between the reactivation attempt and the cancelled date which is not possible.

### Interpretation/Answer

The reactivation initiative by **Call** seems to be the more effective method in trying to win back the customers who have cancelled their recurring bookings. The success rate of the Call initiative is around **53%**, whereas that of Email stands at **35%** and control group initiative at **19%**. 

In [1]:
import pandas as pd
import sqlite3

# Load the CSV data into a DataFrame
df_cancellations = pd.read_csv(r'C:\Users\Rajatth\Downloads\4_cancellations_take_home.csv')
df_emails = pd.read_csv(r'C:\Users\Rajatth\Downloads\3_emails_take_home.csv')
df_calls = pd.read_csv(r'C:\Users\Rajatth\Downloads\2_calls_take_home.csv')
df_appointments = pd.read_csv(r'C:\Users\Rajatth\Downloads\1_cleaning_appointments_take_home.csv')

# Create a SQLite database and connect to it
conn = sqlite3.connect(':memory:')  # This creates a new database in RAM

# Convert the DataFrame into a SQL table
df_cancellations.to_sql('cancellations', conn, if_exists='replace', index=False)
df_calls.to_sql('calls', conn, if_exists='replace', index=False)
df_emails.to_sql('emails', conn, if_exists='replace', index=False)
df_appointments.to_sql('appointments', conn, if_exists='replace', index=False)

# SQL query
query = """
WITH Initiative_Method AS (
    SELECT
        can.customer_id,
        CASE
            WHEN EXISTS (SELECT 1 FROM emails e WHERE e.customer_id = can.customer_id AND email_type = 'reactivation')
                THEN 'email'
            WHEN EXISTS (SELECT 1 FROM calls c WHERE c.customer_id = can.customer_id)
                THEN 'call'
            ELSE 'control'
        END AS initiative_method
    FROM cancellations can
    WHERE NOT EXISTS ( -- Filtering out potentially fake data
        SELECT 1
        FROM (
            SELECT *
            FROM cancellations C
            JOIN appointments A ON C.customer_id = A.customer_id
            JOIN calls CA ON CA.customer_id = C.customer_id
            WHERE A.appointment_timestamp > C.cancellation_timestamp      
                AND A.appointment_timestamp < CA.call_timestamp        -- When call is made after the reactivation 
            UNION ALL
            SELECT *
            FROM cancellations C
            JOIN appointments A ON C.customer_id = A.customer_id
            JOIN emails e ON e.customer_id = C.customer_id
            WHERE A.appointment_timestamp > C.cancellation_timestamp
                AND A.appointment_timestamp < e.email_timestamp          -- When reactivation email is sent after the reactivation
                AND e.email_type = 'reactivation'
        ) AS potentially_fake_data
        WHERE can.customer_id = potentially_fake_data.customer_id
    )
),
reactivated_customers AS (
    SELECT
        can.customer_id,
        CASE
            WHEN EXISTS (SELECT 1
                         FROM appointments app
                         WHERE app.customer_id = can.customer_id
                           AND appointment_timestamp > cancellation_timestamp)
            THEN 'reactivated'
            ELSE 'not_reactivated'
        END AS reactivation_status
    FROM cancellations can
)
SELECT
    initiative_method,
    COUNT(*) AS total_customers_attempted,
    SUM(CASE WHEN reactivation_status = 'reactivated' THEN 1 ELSE 0 END) AS reactivated_customers,
    (SUM(CASE WHEN reactivation_status = 'reactivated' THEN 1 ELSE 0 END) * 100.0 / COUNT(*)) AS success_rate
FROM Initiative_Method
LEFT JOIN reactivated_customers USING (customer_id)
GROUP BY initiative_method;
"""

# Execute the SQL query and fetch the results into a new DataFrame
result_df = pd.read_sql_query(query, conn)

# Display the result
print(result_df)

  initiative_method  total_customers_attempted  reactivated_customers  \
0              call                       1516                    806   
1           control                       1622                    303   
2             email                       1575                    544   

   success_rate  
0     53.166227  
1     18.680641  
2     34.539683  


# *Question 2*: Are the differences between the groups statistically significant?

### Interpretation/Answer

**Yes**, the difference between the 3 different groups are **statistically significant**. 
The approach/method I used for calculating this is the **Chi-Square test**. This method works well without making any assumptions about the data being normally distributed and is flexible.

In order to find the relationship between **initiative_method** and **reactivated_customers**, we need to have an evidence in terms of **chi-square statistic**. This **chi-square statistic** gives a value, which is a measure of how much the counts in the results differ from what we would expect if there is no relationship between **initiative_method** and **reactivated_customers**. The higher this values goes, there is a evidence of strong connection/relationship.

The **p-value** in this Chi-Square test will give us an idea about how likely/unlikely we are, to get the results with respect to the variables as we have observed in the query above. 

I decided to use Chi-Square test because of the **initiative_method** and **reactivated_customers** being *categorical variables*. **initiative_method** has categories: **Call, Control, Email**. **reactivated_customers** basically has categories: **Yes, No**. Here, it helps us to check if these 2 categorical variables are independent of each other. 

The **success_rate** column indicates the proportion of successful reactivations within each initiative type.  The chi-square test is well-suited for comparing proportions across different groups.

### Results of Chi-Square test:

Chi-Square Statistic: 533.6355724231876
P-value: 1.325890853412438e-116
Degrees of Freedom: 2
a potential association between reactivation initiative and success.

The larger the chi-square statistic, the stronger the evidence of a relationship between **initiative_method** and **reactivated_customers**.
The extremely small p-value means it is extremely unlikely that the differences in success rates are due to random chance.
This means there is a strong evidence that the type of initiative **(Call, Control, or Email)**  influences the success rate of reactivations. The **Call** initiative seems to be the **most effective**.



In [59]:
import scipy.stats as stats

# Output table of Question 1: Approach 1 (Same method for Approach 2 as well)
observations = [
    [974, 710],  # Call (Successful, Failed)
    [303, 1319], # Control
    [663, 1031]  # Email
]

# Perform Chi-Square test
result = stats.chi2_contingency(observations)
chi2_stat = result[0]
p_val = result[1]
deg_freedom = result[2]

# Interpretation
if p_val < 0.05:
    conclusion = "A potential association between reactivation initiative and success."
else:
    conclusion = "Insufficient evidence of an association between reactivation initiative and success."

print(f"Chi-Square Statistic: {chi2_stat}")
print(f"P-value: {p_val}")
print(f"Degrees of Freedom: {deg_freedom}")
print(conclusion)

Chi-Square Statistic: 533.6355724231876
P-value: 1.325890853412438e-116
Degrees of Freedom: 2
A potential association between reactivation initiative and success.


# *Question 3*: How to calculate the return on investment for this initiative? What additional information would be needed?

### Interpretation/Answer

To calculate the Return on Investment (ROI) for this customer reactivation initiative, we could use the formula:

**ROI = ((Revenue from Reactivated Customers − Cost of Reactivation Initiatives) / Cost of Reactivation Initiatives ) * 100**

Additional information needed would include: 

* The exact revenue generated from the reactivated customers.
* The costs associated with the reactivation efforts, such as the cost per email cost per call made.
* Any other costs directly related to the reactivation campaign.

This would help to gauge if the revenue generated from the reactivated customers would be more than the costs of the reactivation initiatives or not.



# *Question 4*: How to improve the design of the experiment or collect additional data to make sure we get as many learnings out of the experiment as possible?

### Interpretation/Answer

There are various methods which can be employed to better/improve the experimentation and get informed about many aspects.

* **Additional reactivation methods** - Include additional methods like SMS or in-app notifications. This allows us to compare effectiveness across different channels.
* **A/B Testing** - Within each method (emails, calls), experiment with different subject lines, message content, or call scripts. This helps identify factors driving successful reactivations.
* **Rewards**: Introduce different actions with and without rewards (discounts, free cleaning sessions) to understand their impact on reactivation.
* **Timing for reactivation attempt** - Explore the best timing for reactivation attempts. We can check if the customers respond better closer to their cancellation date or after some time has passed.

* **Customer information** - Include customer demographics and cancellation reasons. This will give an idea about the location specific trends if it exists.
* **Reactivation Reasons** - When a customer reactivates, we can collect information about his reason for getting back. It will offer insights into what was responsible for this.
* **Long term tracking** - We can track reactivated customers for several months after they reactivate. This will help us to understand if they stay loyal or churn quickly.

* **Random assignment** - We should avoid bias in trying to allocate customers to different initiative groups.
* **Good enough sample size** - We need to ensure enough customers are there in each initiative group to get statistically significant results.
* **Clean and consistent data** - The data should be clean and formatted to ensure data analysis can be done effectively.

Using the above mentioned steps we could answer the following potential questions.

**Which reactivation method (email, call, SMS) has the highest success rate?**
**Does reactivation success vary by customer demographics?**
**What types of rewards are most effective?**
**How does the timing between reactivation and future cancellations vary?**

# *Question 5*: How to tell if the data is real or not?

### Interpretation/Answer

The data was found to be fake because of the following reason

* The data contain instances where the cancelled customer has booked a new appointment date for cleaning without getting reactivated through any of the 3 initiatives. The SQL query shows the existence of such records which proves that the dataset is not real. The output contains several records which prove the theory.


In [1]:
import pandas as pd
import sqlite3

# Load the CSV data into a DataFrame
df_cancellations = pd.read_csv(r'C:\Users\Rajatth\Downloads\4_cancellations_take_home.csv')
df_emails = pd.read_csv(r'C:\Users\Rajatth\Downloads\3_emails_take_home.csv')
df_calls = pd.read_csv(r'C:\Users\Rajatth\Downloads\2_calls_take_home.csv')
df_appointments = pd.read_csv(r'C:\Users\Rajatth\Downloads\1_cleaning_appointments_take_home.csv')

# Create a SQLite database and connect to it
conn = sqlite3.connect(':memory:')  # This creates a new database in RAM

# Convert the DataFrame into a SQL table
df_cancellations.to_sql('cancellations', conn, if_exists='replace', index=False)
df_calls.to_sql('calls', conn, if_exists='replace', index=False)
df_emails.to_sql('emails', conn, if_exists='replace', index=False)
df_appointments.to_sql('appointments', conn, if_exists='replace', index=False)

# Prepare your SQL query
# Note that the join is demonstrative and not necessarily supposed to make sense.
query = """
-- This gives the condition for reactivation calls being made after reactivation has been made for the cancelled customers
SELECT *
FROM cancellations C
JOIN appointments A
     ON C.customer_id = A.customer_id
JOIN calls CA
     ON CA.customer_id = C.customer_id
WHERE A.appointment_timestamp > C.cancellation_timestamp 
      AND A.appointment_timestamp < CA.call_timestamp AND CA.call_answered = 'FALSE'
      
UNION ALL 

-- This gives the condition for reactivation emails being sent after reactivation has been made for the cancelled customers
SELECT *
FROM cancellations C
JOIN appointments A
     ON C.customer_id = A.customer_id
JOIN emails e
     ON e.customer_id = C.customer_id
WHERE A.appointment_timestamp > C.cancellation_timestamp 
      AND A.appointment_timestamp < e.email_timestamp AND e.email_type = 'reactivation';
"""

# Execute the SQL query and fetch the results into a new DataFrame
result_df = pd.read_sql_query(query, conn)

# Display the result
print(result_df)

     customer_id cancellation_timestamp  customer_id appointment_timestamp  \
0             86             2023-01-25           86            2023-02-14   
1            195             2023-01-17          195            2023-02-09   
2            207             2023-01-25          207            2023-02-05   
3            227             2023-01-27          227            2023-02-08   
4            277             2023-01-12          277            2023-02-04   
..           ...                    ...          ...                   ...   
129         4848             2023-01-22         4848            2023-02-05   
130         4889             2023-01-19         4889            2023-02-03   
131         4889             2023-01-19         4889            2023-02-03   
132         4934             2023-01-31         4934            2023-02-17   
133         4973             2023-01-27         4973            2023-02-10   

     customer_id call_timestamp call_answered  
0             8